<a href="https://colab.research.google.com/github/aboelela924/Deep-learning-with-TF2-and-Keras/blob/master/Deep_learning_with_TF2_and_Keras_chapter_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec
import os

In [ ]:
if not os.path.isdir("/content/data"):
    os.mkdir("/content/data")

In [ ]:
dataset = api.load("text8")
model = Word2Vec(dataset)

model.save("data/text8-word2vec.bin")

[==================================================] 100.0% 31.6/31.6MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load("/content/data/text8-word2vec.bin")
word_vectors = model.wv

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
words = word_vectors.vocab.keys()
print([word for i, word in enumerate(words) if i < 10])
assert "king" in words

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [ ]:
print(word_vectors.most_similar("king"))

[('prince', 0.7400096654891968), ('emperor', 0.7214909195899963), ('throne', 0.7093262672424316), ('queen', 0.7061789035797119), ('kings', 0.6803011894226074), ('aragon', 0.675293505191803), ('vii', 0.670933723449707), ('regent', 0.6631604433059692), ('pope', 0.6609975099563599), ('elector', 0.6554006934165955)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
def print_most_similar_words(word_score_pairs, c):
    for i, (word, similar_word_score) in enumerate(word_score_pairs):
        print("{:.3f} {:s}".format(similar_word_score, word))
        if i >= c:
            break
print_most_similar_words(word_vectors.most_similar("king"), 5)

0.740 prince
0.721 emperor
0.709 throne
0.706 queen
0.680 kings
0.675 aragon


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print_most_similar_words(word_vectors.most_similar(positive=["france", "berlin"], negative=["paris"]), 0)

0.807 germany


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print_most_similar_words(word_vectors.most_similar_cosmul(positive=["france", "berlin"], negative=["paris"]), 0)

0.984 germany


In [ ]:
print(word_vectors.doesnt_match(["berlin", "england", "rome","madrid"]))

england


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(word_vectors.similar_by_word("rice"))

[('beef', 0.8637088537216187), ('pork', 0.8619951009750366), ('potatoes', 0.8536219596862793), ('tomato', 0.8464804887771606), ('citrus', 0.8418481349945068), ('barley', 0.8387696743011475), ('soy', 0.8323472738265991), ('cereal', 0.8316136598587036), ('onions', 0.8302257657051086), ('peanuts', 0.828937828540802)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<h1>Using word embeddings for spam detection</h1>

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=6fc4b00eabbe09eda19c685aefff88639e452f200b66a4ea6d95dd90004faed3
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import argparse 
import gensim.downloader as api 
import numpy as np 
import os 
import shutil 
import tensorflow as tf
import wget
import zipfile

from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
if not os.path.isdir("/content/datasets"):
    os.mkdir("/content/datasets")
    os.mkdir("/content/datasets/SMSSpamCollection")

In [ ]:
def download_and_read(url):
    local_file = url.split("/")[-1]
    wget.download(url, f"/content/{local_file}")

    with zipfile.ZipFile(f"/content/{local_file}", 'r') as zip_ref:
        zip_ref.extractall("/content")
    labels, texts = [], []
    path = f"/content/SMSSpamCollection"
    with open(path, "r") as fin:
        for line in fin:
            label, text = line.strip().split("\t")
            labels.append(1 if label == "spam" else 0)
            texts.append(text)
    return texts, labels

In [ ]:
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
texts, labels = download_and_read(DATASET_URL)

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences)

print(f"num of sequences {len(text_sequences)}")
print(f"length of the max sequence {len(text_sequences[0])}")

num of sequences 5574
length of the max sequence 189


In [ ]:
NUM_CLASSES = 2
labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

In [ ]:
print(labels[:10])

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [ ]:
word2idx = tokenizer.word_index
idx2word = {value: key for key, value in word2idx.items()}
word2idx["PAD"] = 0
idx2word[0] = "PAD"
vocab_size = len(word2idx)
print("Vocab size: {:d}".format(vocab_size))

Vocab size: 9010


In [ ]:
dataset_size = len(text_sequences)

dataset = tf.data.Dataset.from_tensor_slices((text_sequences, labels))
dataset = dataset.shuffle(1000)

test_size = dataset_size // 4
val_size = dataset_size // 10

val_dataset = dataset.take(val_size)
test_dataset = dataset.skip(val_size).take(test_size)
train_dataset = dataset.skip(val_size+test_size)

In [ ]:
BATCH_SIZE = 128
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_embedding_matrix(sequences, word2idx, embedding_dim, embedding_file):
    if os.path.exists(embedding_file):
        E = np.load(embedding_file)
    else:
        vocab_size = len(word2idx)
        E = np.zeros((vocab_size, embedding_dim))
        word_vectors = api.load(EMBEDDING_MODEL)
        for word, idx in word2idx.items():
            try:
                E[idx] = word_vectors.word_vec(word)
            except KeyError:
                pass
        np.save(embedding_file, E)
    return E

In [ ]:
EMBEDDING_DIM = 300
DATA_DIR = "/content/data"
EMBEDDING_NUMPY_FILE = os.path.join(DATA_DIR, "E.npy")
EMBEDDING_MODEL = "glove-wiki-gigaword-300"
E = build_embedding_matrix(text_sequences, word2idx, EMBEDDING_DIM, EMBEDDING_NUMPY_FILE)

In [ ]:
class SpamClassifierModel(tf.keras.Model):
    def __init__(self, vocab_size, embedd_size, input_length, num_filters, 
                 kernel_size, output_size, run_mode, embedding_weights, **kwargs):
        super(SpamClassifierModel, self).__init__(**kwargs)

        if run_mode == "scratch":
            self.embedding = tf.keras.layers.Embedding(vocab_size, 
                                                       embedd_size, 
                                                       input_length=input_length, 
                                                       trainable=False)
        elif run_mode == "vectorizer":
            self.embedding = tf.keras.layers.Embedding(vocab_size, 
                                                       embedd_size,
                                                       input_length=input_length,
                                                       weights=[embedding_weights], 
                                                       trainable = False)
        else:
            self.embedding = tf.keras.layers.Embedding(vocab_size, 
                                                       embedd_size,
                                                       input_length=input_length,
                                                       weights=[embedding_weights], 
                                                       trainable = True)
            
        self.conv = tf.keras.layers.Conv1D(filters=num_filters, 
                                           kernel_size=kernel_size, 
                                           activation="relu")
        self.dropout = tf.keras.layers.SpatialDropout1D(0.2)
        self.pool = tf.keras.layers.GlobalMaxPooling1D()
        self.dense = tf.keras.layers.Dense(output_size, activation="softmax")
    def call(self, x):
        x = self.embedding(x)
        x = self.conv(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.dense(x)
        return x


In [ ]:
conv_num_filters = 256
conv_kernel_size = 3
run_modes = ["scratch", "vectorizer", "other"]
model = SpamClassifierModel(vocab_size, EMBEDDING_DIM, len(text_sequences[0]), 
                            conv_num_filters, conv_kernel_size, 2, run_modes[1], 
                            E)
model.build(input_shape=(None, len(text_sequences[0])))

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
EPOCHS = 3
CLASS_WEIGHTS = {0:1, 1:8}
model.fit_generator(train_dataset, epochs=EPOCHS, validation_data=val_dataset, 
                    class_weight=CLASS_WEIGHTS)

Epoch 1/3
28/28 [==============================] - 13s 466ms/step - loss: 0.5121 - accuracy: 0.8708 - val_loss: 0.1355 - val_accuracy: 0.9609
Epoch 2/3
28/28 [==============================] - 13s 465ms/step - loss: 0.1801 - accuracy: 0.9623 - val_loss: 0.0873 - val_accuracy: 0.9668
Epoch 3/3
28/28 [==============================] - 13s 461ms/step - loss: 0.1059 - accuracy: 0.9844 - val_loss: 0.0489 - val_accuracy: 0.9883


In [ ]:
labels, predictions = [], []
for Xtest, Ytest in test_dataset:
    Ytest_ = model.predict_on_batch(Xtest)
    ytest = np.argmax(Ytest, axis=1)
    ytest_ = np.argmax(Ytest_, axis=1)
    labels.extend(ytest.tolist())
    predictions.extend(ytest_.tolist())
print("test accuracy: {:.3f}".format(accuracy_score(labels,predictions)))
print("confusion matrix")
print(confusion_matrix(labels, predictions))

test accuracy: 0.987
confusion matrix
[[1091   14]
 [   3  172]]


<h1>Node2Vec and DeepWalk</h1>

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=11fe8b3fd1e86c1109d3df5ecf1131e5c3a26233743d8b5d4b13f8f508c8ab70
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import gensim
import logging
import numpy as np
import shutil
import tensorflow as tf
import os
import wget
import zipfile

from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import  cosine_similarity

In [ ]:
if not os.path.isdir("/content/data"):
    os.mkdir("/content/data")

In [ ]:
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

In [ ]:
DATA_DIR = "/content/data"
UIC_DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00371/NIPS_1987-2015.csv"

In [ ]:
def download_and_read(url):
    local_file = url.split("/")[-1]
    wget.download(url, f"/content/{local_file}")

    # with zipfile.ZipFile(f"/content/{local_file}", 'r') as zip_ref:
    #     zip_ref.extractall("/content")
    row_ids, col_ids, data = [], [], []
    rid = 0
    i = 0
    with open(f"/content/{local_file}", 'r') as reader:
        for line in reader:
            line = line.strip()
            if line.startswith("\"\","):
                continue
            counts = np.array([int(x) for x in line.split(",")[1:]])
            nz_col_ids = np.nonzero(counts)[0]
            nz_data = counts[nz_col_ids]
            nz_rows_ids = np.repeat(rid, len(nz_col_ids))
            rid += 1
            row_ids.extend(nz_rows_ids)
            col_ids.extend(nz_col_ids)
            data.extend(nz_data)
    reader.close()
    TD = csr_matrix((
        np.array(data), (
            np.array(row_ids), np.array(col_ids)
        )
    ), shape=(rid, counts.shape[0]))
    return TD

In [ ]:
TD = download_and_read(UIC_DATA_URL)
E = TD.T * TD
E[E>0] = 1

In [22]:
TD2 = download_and_read(UIC_DATA_URL)
E2 = TD.T * TD

In [ ]:
# NUM_WALKS_PER_VERTEX = 32
# MAX_PATH_LEGNTH = 40
# RESTART_PROB = 0.15

# RANDOM_WALK_FILE = os.path.join(DATA_DIR, "random-walk.txt")

# def construct_walks(E, n , alpha, ofile):
#     if os.path.exists(ofile):
#         print("Random walks generated already, skipping.")
#         return
#     with open(f"/content/{local_file}", 'r') as reader:
#         for i in range(E.shape[0]):
#             if i % 100 == 0:
#                 print("{:d} random walks generated from {:d} vertices".foramt(n *i, i))
#             for j in range(n):
#                 curr = i 
#                 walk = [curr]
#                 target_nodes = np.nonzeros(E[curr])[0]
#                 for k in range()